get data for BAC


In [ ]:
%%capture
pip install git+https://github.com/femtotrader/python-eodhistoricaldata.git


In [ ]:
%%capture 
pip install pulp

In [ ]:
%%capture
pip install ipywidgets


In [ ]:
%%capture
!jupyter nbextension enable --py widgetsnbextension


In [ ]:
lnk =  'https://eodhistoricaldata.com/api/options/BAC.US?api_token=' 
token = ''
%env EOD_HISTORICAL_API_KEY=token


env: EOD_HISTORICAL_API_KEY=token


In [ ]:
%matplotlib widget
import requests
import datetime
import pandas as pd
from pandas.api.types import is_number
from urllib.parse import urlencode
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

def _init_session(session):
    """
    Returns a requests.Session (or CachedSession)
    """
    if session is None:
        return requests.Session()
    return session


def _url(url, params):
    """
    Returns long url with parameters
    http://mydomain.com?param1=...&param2=...
    """
    if params is not None and len(params) > 0:
        return url + "?" + urlencode(params)
    else:
        return url


class RemoteDataError(IOError):
    """
    Remote data exception
    """
    pass


def _format_date(dt):
    """
    Returns formated date
    """
    if dt is None:
        return dt
    return dt.strftime("%Y-%m-%d")


def _sanitize_dates(start, end):
    """
    Return (datetime_start, datetime_end) tuple
    """
    if is_number(start):
        # regard int as year
        start = datetime.datetime(start, 1, 1)
    start = pd.to_datetime(start)

    if is_number(end):
        # regard int as year
        end = datetime.datetime(end, 1, 1)
    end = pd.to_datetime(end)

    if start is not None and end is not None:
        if start > end:
            raise Exception("end must be after start")

    return start, end

In [ ]:

import os
import requests
import pandas as pd
from io import StringIO
import json
# from ._utils import (_init_session, _format_date,
#                      _sanitize_dates, _url, RemoteDataError)

EOD_HISTORICAL_DATA_API_KEY_ENV_VAR = "EOD_HISTORICAL_API_KEY"
EOD_HISTORICAL_DATA_API_KEY_DEFAULT = "OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX"
EOD_HISTORICAL_DATA_API_URL = "https://eodhistoricaldata.com/api"

pd.set_option("max_rows", 10)

def get_options_data(symbol, exchange, start=None, end=None,
                 api_key=token,
                 session=None):
    """
    Returns EOD (end of day data) for a given symbol
    """
    symbol_exchange = symbol + "." + exchange
    session = _init_session(session)
    start, end = _sanitize_dates(start, end)
    endpoint = "/options/{symbol_exchange}".format(symbol_exchange=symbol_exchange)
    url = EOD_HISTORICAL_DATA_API_URL + endpoint
    params = {
        "api_token": api_key,
        "from": _format_date(start),
        "to": _format_date(end)
    }
    r = session.get(url, params=params)
    if r.status_code == requests.codes.ok:
        df = json.loads(r.text)
        return df
    else:
        params["api_token"] = "YOUR_HIDDEN_API"
        raise RemoteDataError(r.status_code, r.reason, _url(url, params))
        
def get_stock_data(symbol, exchange, start=None, end=None,
                 api_key=token,
                 session=None):
    """
    Returns EOD (end of day data) for a given symbol
    """
    symbol_exchange = symbol + "." + exchange
    session = _init_session(session)
    start, end = _sanitize_dates(start, end)
    endpoint = "/eod/{symbol_exchange}".format(symbol_exchange=symbol_exchange)
    url = EOD_HISTORICAL_DATA_API_URL + endpoint
    params = {
        "api_token": api_key,
        "from": _format_date(start),
        "to": _format_date(end)
    }
    r = session.get(url, params=params)
    if r.status_code == requests.codes.ok:
        df = pd.read_csv(StringIO(r.text), skipfooter=1,
                         parse_dates=[0], index_col=0)
        return df
    else:
        params["api_token"] = "YOUR_HIDDEN_API"
        raise RemoteDataError(r.status_code, r.reason, _url(url, params))

In [ ]:
symbol = 'WFC'
exchange = 'US'

In [ ]:

pd.options.display.width = 0
df_opts = get_options_data(symbol, exchange, start=None, end=None,
                       
                       
                 api_key=token,
                 session=None)
df_stock = get_stock_data(symbol, exchange, api_key=token, session=None)
df_stock['MA20'] = df_stock['Close'].rolling(window=20).mean()
df_stock['20dSTD'] = df_stock['Close'].rolling(window=20).std() 


df_stock['Upper_MA20'] = df_stock['MA20'] + (df_stock['20dSTD'] * 2)
df_stock['Lower_MA20'] = df_stock['MA20'] - (df_stock['20dSTD'] * 2)
df_stock['Upper_current'] = df_stock['Close'] + (df_stock['Upper_MA20'] -df_stock['MA20'])
df_stock['Lower_current'] = df_stock['Close'] - (df_stock['MA20'] -df_stock['Lower_MA20'])
df_stock['Boli_int'] = (df_stock['Upper_current'] - df_stock['Lower_current'])
# df_stock['Boli_int2'] = df_stock['Boli_int'].fillna(0).astype('int')

# df_stock.tail()
current_spot = df_stock['Close'].iloc[-1]

<ipython-input-7-1820381762a6>:57: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(StringIO(r.text), skipfooter=1,


In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
def get_init_df(df_opts):
    df = df_opts
    df = pd.json_normalize(df, 'data', sep='_').drop(['expirationDate', 'impliedVolatility'], axis=1)
    # print(df.head(2))
    df3 = df.explode('options_CALL').drop('options_PUT', axis = 1)
    df3 = df3.rename( columns={'options_CALL': "options"})
    df3.head(2)
    df4 = pd.concat([df3.drop(['options'], axis=1), df3['options'].apply(pd.Series)], axis=1)
    df3 = df.explode('options_PUT').drop('options_CALL', axis = 1)
    df3 = df3.rename( columns={'options_PUT': "options"})
    df3.head(2)
    df5 = pd.concat([df3.drop(['options'], axis=1), df3['options'].apply(pd.Series)], axis=1)
    # df = df4.append(df5) 
    df = pd.concat([df4, df5], ignore_index = True)
    df['expirationDate'] = df['expirationDate'].astype('datetime64[ms]')
    # df = df[~df.index.duplicated(keep='first')]
    # df = df[df['expirationDate'] == '2021-09-21' ]
    # df = df[df['type'] == 'CALL' ]
    # df = df[ (df['strike'] >79) & (df['strike'] <135)  ]
    print(df.shape)
    df.sort_values(by=['contractName']).head(1)
    return df

In [ ]:
import numpy as np
from scipy.stats import norm

def call_payoff(sT, strike_price, premium):
    return np.where(np.array(sT) > strike_price, np.array(sT) - strike_price, 0) - premium

def put_payoff(sT, strike_price, premium):
    return np.where(sT < strike_price, strike_price - np.array(sT), 0) - premium

def call_bs_payoff(sT, strike_price, bs, premium, items):
    cp = items/np.abs(items)
#     print(cp)
    a= np.where(np.array(sT) > strike_price, np.array(sT) - strike_price, 0)
#     print('bs')
#     print(bs)
#     print('payoff')
#     print(np.where(np.array(sT) > strike_price, np.array(sT) - strike_price, 0))
    a=(np.add((cp)*bs, (-cp)*premium) )
#            lc     items:1  bs:3  premium:3  
#            lc 60days    1  bs:6    6-3
#            sc      items:-1 bs:3 premium:3
#                             bs 6  -(6-3) -6 + 3
#     print(a.shape)
    return a

def put_bs_payoff(sT, strike_price, bs, premium, items):
#     return np.subtract(np.where(np.array(sT) < strike_price, strike_price - np.array(sT), 0) , bs)
    cp = items/np.abs(items)
#     print(cp)
    a=(np.add((cp)*bs, (-cp)*premium) )
    return a

# call_payoff([39, 60], 50, [[1.4, 5.6], [ 7.7, 5.6], [1.3, 4.5]])
# a=call_bs_payoff(bought['sT'][0], bought['strike_price'][0], bought['bs'][0][161], bought['avg_price'][0], bought['items'][0] )
# a
# call_payoff([39, 60], 50, [[1.4, 5.6], [ 7.7, 5.6], [1.3, 4.5]])



In [ ]:
# S is the spot price SLIDER HERE
# K is the strike price 
# T is the fraction of days to strike date divided by 252 (stock market days) SLIDER HERE
# r is the risk free rate 0.01
# sigma is the annual volatility of the returns of the stock

# Task 1 : create sanity check : use B&S to generate theortical prices and cross check with actual data and report discrepancies
# Task 2 : use implied volatility in our own calculations

# # def black_scholes(S, K, T, r, sigma):
# def black_scholes(row):
#     d1 = np.log(row.S/(row.K/(1 + row.r)**row.T)/(row.sigma*sqrt(row.T))) + (row.sigma*sqrt(row.T))/2
#     d2 = d1 - row.sigma * np.sqrt(row.T)
#     return row.S * norm.cdf(d1) - (row.K/(1 + row.r)**row.T) * norm.cdf(d2)

In [ ]:
from scipy import stats
import math

def black_scholes (cp, s, k, t, v, rf, div):
        """ Price an option using the Black-Scholes model.
        s: initial stock price
        k: strike price
        t: expiration time
        v: volatility
        rf: risk-free rate
        div: dividend
        cp: +1/-1 for call/put
        """

#         d1 = (math.log(s/k)+(rf-div+0.5*math.pow(v,2))*t)/(v*math.sqrt(t))
#         d2 = d1 - v*math.sqrt(t)
            
#         optprice = (cp*s*math.exp(-div*t)*stats.norm.cdf(cp*d1)) - (cp*k*math.exp(-rf*t)*stats.norm.cdf(cp*d2))
#         d1 = (np.log(np.array(s)/k) + (rf+0.5*math.pow(v,2))*np.array(t))/(v*math.sqrt(np.array(t)))
        d11 = np.log(np.array([s])/k)
#         print(d11.shape)
        d12 = (rf+0.5*math.pow(v,2))*np.array([t])
        d13 = (v*np.sqrt([t])).T
#         print(np.add(d11, d12.T) )
        d1 = np.add(d11, d12.T) / d13
#         print( d1 )
        d2 = np.subtract(d1, d13)
        optprice11 = (cp*np.array([s])*np.exp(-div*np.array([t]).T)*stats.norm.cdf(cp*d1))
        optprice12 = (cp*k*np.exp(-rf*np.array([t]).T)*stats.norm.cdf(cp*d2))
        d = (optprice11-optprice12)
#         print( optprice11-optprice12 )
#         + (rf+0.5*math.pow(v,2))*np.array(t))/(v*math.sqrt(np.array(t)))
#         print(optprice11.shape)
        return d

def black_scholes2(row):
    if row['option']=='C':
        cp=1
    else:
        cp=-1
#     print(black_scholes (cp, row.S, row.K, row['T'], row.sigma/100, row.r, 0))
    return black_scholes (cp, row.S, row.K, row['T'], row.sigma/100, row.r, 0)

# print(black_scholes (1, bought['sT'][0], 50, bought['T'][0][161], 32.3085/100, 0.002, 0)   )
# df[df['contractName']=='WFC210917C00050000']
# bought['T'][0].shape
# k=0
# a = call_bs_payoff(bought['sT'][k], bought['strike_price'][k], black_scholes(1, bought['sT'][k], bought.strike_price[k], bought['T'][k], bought.sigma[k], bought.r[k], 0))
# a[152]
# df[df['contractName']=='WFC210917C00050000']
# b=black_scholes(1, bought['sT'][k], bought.strike_price[k], bought['T'][k], bought.sigma[k]/100, bought.r[k], 0)
# b[161]
# bought['sT'][0].shape
# bs = black_scholes (1, bought['sT'][0], 50, bought['T'][0][161], 32.3085/100, 0.002, 0)
# bs[0][39]

In [ ]:
# Bring the theoretical price from previous dataset
from math import log, sqrt, pi, exp
from scipy.stats import norm
import re
pd.set_option('display.max_rows', None)

bought_real = [['WFC210917C00055000', -5, 0.68],
          ['WFC210917C00052500', 4, 1.23],
['WFC210917C00050000', -4, 1.12],
['WFC210917C00047500', 2, 1.46],
['WFC210917C00042500', 1, 3.66],
['WFC210917P00030000', -1, 0.46],
['WFC210917P00045000', 1, 3.1],
              ['WFC210917P00042500', -1, 2.01],
              ['WFC210917P00037500', 1, 0.8],
              ['WFC210917P00025000', 1, 0.45],
              ['WFC210917P00022500', -3, 0.47]]
df = get_init_df(df_opts)
strike_max = 2*int(df.strike.max())

premium = 0

def get_bought(bought_real, df, strike_max):
    bought = pd.DataFrame(bought_real, columns = ['contractName', 'items', 'avg_price'])
    bought['symbol'] = bought['contractName'].str.extract(r'(?P<Str>\w[^\d]+)', expand=False)
    bought['expirationDate'] = bought['contractName'].str.extract('(?P<Str>\d{6})', expand=True).astype('datetime64[ms]')
    bought['option'] = bought['contractName'].str.extract(r'\d+([CP])\d+', expand=False)
    bought['strike_price'] = bought['contractName'].str.extract(r'(\d+)$', expand=False).astype('float')/1000
    # print(bought)
    bought['sT'] = bought.strike_price.apply(lambda x: np.arange(1, np.int(strike_max),1) )
    bought['cp'] = bought.option.apply(lambda x: 1 if x=='call' else -1 )
    bought['optionPayoff'] =bought.apply(lambda x: 100*x['items']*call_payoff (x['sT'], x['strike_price'], x['avg_price']) if x['option']=='C'  \
                                         else 100*x['items']*put_payoff (x['sT'], x['strike_price'], x['avg_price']), axis = 1)
    bought = pd.merge(bought, df, how = 'left', on = 'contractName') 
    # print(bought)
    bought['theor_optPayoff'] =bought.apply(lambda x: 100*x['items']*call_payoff (x['sT'], x['strike_price'], x['theoretical']) if x['option']=='C'  \
                                         else 100*x['items']*put_payoff (x['sT'], x['strike_price'], x['theoretical']), axis = 1)
    bought['T'] = bought.daysBeforeExpiration.apply(lambda x: (np.arange(1,np.int(x),1)/365) )
    bought['days_array'] = bought.daysBeforeExpiration.apply(lambda x: (np.arange(1,np.int(x),1)) )

    bought['K'] = bought['strike_price']
    bought['S'] = bought['sT']
    bought['r'] = 0.002
    bought['sigma'] = bought['impliedVolatility'].astype('float')
    bought['option_type'] = bought['type'].str.lower()

    bought['bs'] = bought.apply(black_scholes2, axis=1)
    bought['bs_Payoff'] =bought.apply(lambda x: 100*np.abs(x['items'])*call_bs_payoff (x['sT'], x['strike_price'], x['bs'],x['avg_price'], x['items']  ) if x['option']=='C'  \
                                         else 100*np.abs(x['items'])*put_bs_payoff (x['sT'], x['strike_price'], x['bs'], x['avg_price'], x['items'] ), axis = 1)
    payoff_straddle = np.sum(bought.optionPayoff)
    theor_payoff_straddle = np.sum(bought.theor_optPayoff)
    bb =np.stack(bought['bs_Payoff'])
    # print(bought['bs_Payoff'][0][161])
    b_s_payoff_straddle = np.sum(bb, axis=0)
    return bought, b_s_payoff_straddle





(640, 34)


In [ ]:
## Get all strikes in dataframe
## calculate for each bs price in (spot_min, spot_max)
## 
## union with bought
# print(bought.expirationDate_x[0])
def build_proposal(df, bought, it, strike_max):
    df['symbol'] = df['contractName'].str.extract(r'(?P<Str>\w[^\d]+)', expand=False)
    df['expirationDate'] = df['contractName'].str.extract('(?P<Str>\d{6})', expand=True).astype('datetime64[ms]')
    df=df[df['expirationDate']==bought.expirationDate_x[0]]
    df['option'] = df['contractName'].str.extract(r'\d+([CP])\d+', expand=False)
    df['strike_price'] = df['contractName'].str.extract(r'(\d+)$', expand=False).astype('float')/1000
    df['sT'] = df.strike_price.apply(lambda x: np.arange(1, np.int(strike_max),1) )
    df['items'] = df.strike_price.apply(lambda x: np.linspace(-it, it, 2*it+1 , endpoint=True) )
    df['cp'] = df.option.apply(lambda x: 1 if x=='call' else -1 )

    # df = df.iloc[3:4]
    # df = pd.merge(df, df, how = 'left', on = 'contractName') 
    # print(df)

    df['T'] = df.daysBeforeExpiration.apply(lambda x: (np.arange(1,np.int(x),1)/365) )
    df['days_array'] = df.daysBeforeExpiration.apply(lambda x: (np.arange(1,np.int(x),1)) )

    df['K'] = df['strike_price']
    df['S'] = df['sT']
    df['r'] = 0.002
    df['sigma'] = df['impliedVolatility'].astype('float')
    df['option_type'] = df['type'].str.lower()
    df['bs'] = df.apply(black_scholes2, axis=1)
#     df['bs_Payoff'] =df.apply(lambda x: call_fut_payoff (x['sT'], x['strike_price'], x['bs'],x['theoretical'], 1, x['cp']  ) if x['option']=='C'  \
#                                          else put_fut_payoff (x['sT'], x['strike_price'], x['bs'], x['theoretical'], 1, x['cp'] ), axis = 1)
    df['long_pl'] =df.apply(lambda x: call_fut_payoff (x['sT'], x['strike_price'], x['bs'],x['theoretical'], 1,   ), axis = 1)
    df['short_pl'] =df.apply(lambda x: call_fut_payoff (x['sT'], x['strike_price'], x['bs'],x['theoretical'], -1  ) , axis = 1)
    return df

def build_second_proposal(df, bought, it, strike_max, pidx, didx):
    df['symbol'] = df['contractName'].str.extract(r'(?P<Str>\w[^\d]+)', expand=False)
    df['expirationDate'] = df['contractName'].str.extract('(?P<Str>\d{6})', expand=True).astype('datetime64[ms]')
    df=df[df['expirationDate']==bought.expirationDate_x[0]]
    df['option'] = df['contractName'].str.extract(r'\d+([CP])\d+', expand=False)
    df['strike_price'] = df['contractName'].str.extract(r'(\d+)$', expand=False).astype('float')/1000
    df['sT'] = df.strike_price.apply(lambda x: np.arange(1, np.int(strike_max),1) )
    df['items'] = df.strike_price.apply(lambda x: np.linspace(-it, it, 2*it+1 , endpoint=True) )
    df['cp'] = df.option.apply(lambda x: 1 if x=='call' else -1 )

    # df = df.iloc[3:4]
    # df = pd.merge(df, df, how = 'left', on = 'contractName') 
    # print(df)

    df['T'] = df.daysBeforeExpiration.apply(lambda x: (np.arange(1,np.int(x),1)/365) )
    df['days_array'] = df.daysBeforeExpiration.apply(lambda x: (np.arange(1,np.int(x),1)) )

    df['K'] = df['strike_price']
    df['S'] = df['sT']
    df['r'] = 0.002
    df['sigma'] = df['impliedVolatility'].astype('float')
    df['option_type'] = df['type'].str.lower()
    df['bs'] = df.apply(black_scholes2, axis=1)
#     df['bs_Payoff'] =df.apply(lambda x: call_fut_payoff (x['sT'], x['strike_price'], x['bs'],x['theoretical'], 1, x['cp']  ) if x['option']=='C'  \
#                                          else put_fut_payoff (x['sT'], x['strike_price'], x['bs'], x['theoretical'], 1, x['cp'] ), axis = 1)
    df['long_pl'] =df.apply(lambda x: call_fut_payoff (x['sT'], x['strike_price'], x['bs'],x['bs'][didx][pidx], 1,   ), axis = 1)
    df['short_pl'] =df.apply(lambda x: call_fut_payoff (x['sT'], x['strike_price'], x['bs'],x['bs'][didx][pidx], -1  ) , axis = 1)
    return df

def put_fut_payoff(sT, strike_price, bs, premium, ls):
#     return np.subtract(np.where(np.array(sT) < strike_price, strike_price - np.array(sT), 0) , bs)
#     cp = np.nan_to_num(items/np.abs(items))
#     print(cp)
    a=(np.add(ls*bs, (-ls)*premium) )
    return a

def call_fut_payoff(sT, strike_price, bs, premium, ls):
#     cp = np.nan_to_num(items/np.abs(items))
#     print(cp)
#     a= np.where(np.array(sT) > strike_price, np.array(sT) - strike_price, 0)
#     print('bs')
#     print(bs)
#     print('payoff')
#     print(np.where(np.array(sT) > strike_price, np.array(sT) - strike_price, 0))
#     print((-cp)*premium)
#     print((np.array(cp[:,None, None])*bs).shape)
    a=(np.add(ls*bs, (-ls)*premium) )
#     print(a.shape)
    return a

# df =  build_proposal(df, bought, 4)

# call_fut_payoff (df['sT'].iloc[0], df['strike_price'].iloc[0], df['bs'].iloc[0], df['theoretical'].iloc[0], df['items'].iloc[0])


In [ ]:
import warnings
warnings.filterwarnings('ignore')
from scipy.optimize import minimize
from scipy.integrate import simps
from scipy import integrate
from numpy import trapz
# spot_min = 10
# spot_max = 70


def getaux(df, bought, it, spot_min, spot_max, dbe, b_s_payoff_straddle):
    half_dbe = dbe//2
    df = build_proposal(df, bought, it, strike_max)
    dfpl=np.stack(df['long_pl'])
    dfps=np.stack(df['short_pl'])
    dfpl = np.moveaxis(dfpl, 1, 0)
    dfps = np.moveaxis(dfps, 1, 0)
    pc_l = dfpl[half_dbe][:,spot_min-1:spot_max]
    pc_s = dfps[half_dbe][:,spot_min-1:spot_max]
    pc2_l = dfpl[dbe][:,spot_min-1:spot_max]
    pc2_s = dfps[dbe][:,spot_min-1:spot_max]
    pc_final_l = dfpl[1][:,spot_min-1:spot_max]
    pc_final_s = dfps[1][:,spot_min-1:spot_max]
    pos = b_s_payoff_straddle[half_dbe][spot_min-1:spot_max]
    pos_final = b_s_payoff_straddle[1][spot_min-1:spot_max]
    pos2 = b_s_payoff_straddle[dbe][spot_min-1:spot_max]
    return df, dfpl, dfps, pc_l, pc_s, pos, pc2_l, pc2_s, pc_final_l, pc_final_s, pos_final, pos2

def get_future_proposals(df, bought, it, spot_min, spot_max, dbe,b_s_payoff_straddle, pidx, didx):
    half_dbe = dbe//2
    df = build_second_proposal(df, bought, it, strike_max, pidx, didx)
    dfpl=np.stack(df['long_pl'])
    dfps=np.stack(df['short_pl'])
    dfpl = np.moveaxis(dfpl, 1, 0)
    dfps = np.moveaxis(dfps, 1, 0)
    pc2_l = dfpl[dbe][:,spot_min-1:spot_max]
    pc2_s = dfps[dbe][:,spot_min-1:spot_max]
    pc_final_l = dfpl[1][:,spot_min-1:spot_max]
    pc_final_s = dfps[1][:,spot_min-1:spot_max]
    pos = b_s_payoff_straddle[half_dbe][spot_min-1:spot_max]
    pos_final = b_s_payoff_straddle[1][spot_min-1:spot_max]
    pos2 = b_s_payoff_straddle[dbe][spot_min-1:spot_max]
    return df, dfpl, dfps, pc_l, pc_s, pos, pc2_l, pc2_s, pc_final_l, pc_final_s, pos_final, pos2

def optimize_fn(df, bought, it,b_s_payoff_straddle, spot_min, spot_max):
    
    
    print(pc.shape)
    short_lim = df[['strike_price', 'option']]
    short_lim['current_spot'] = df_stock['Close'].iloc[-1]
    short_lim['short_lim'] = short_lim.apply(lambda x: -1 if (x.current_spot<x.strike_price and x.option=='C')\
                                             or (x.current_spot>x.strike_price and x.option=='P') else 0, axis =1)
    short_lim['call_lim'] = 1
    short_lim['lim_bnds'] = short_lim.apply(lambda x: np.array([x.short_lim , x.call_lim]) , axis=1)
    max_items_per_contract = it
    bnds = np.stack(max_items_per_contract*np.array(short_lim.lim_bnds))
    cons = (
        
        {'type': 'ineq', 'fun': lambda x:  2-np.where((x<0.5) & (x>0.1),1,0).sum() },
        {'type': 'ineq', 'fun': lambda x:  2-np.where((x<-0.1) & (x>-0.5),1,0).sum() },
        {'type': 'ineq', 'fun': lambda x:  4 -   np.abs(np.round(x,2)).sum()},
               {'type': 'ineq', 'fun': lambda x:  10-(np.round(x[:,None],0)*df.theoretical[:,None]).sum(0)},
           {'type': 'ineq', 'fun': lambda x:  4 + np.round(np.where(x>0,x,0),0).sum(0)+ np.round(np.where(x<0,x,0),0).sum(0)}
           )                                  
    result = minimize(objective, np.zeros(pc.shape[0]), method='trust-constr', options={ 'xtol': 1e-08, \
                                                                                        'gtol': 1e-08, 
                                                                                        'barrier_tol': 1e-06,
                                                                                        'sparse_jacobian': None, 
                                                                                        'maxiter': 100000,
                                                                                        'disp':True}, \
                      bounds = bnds, constraints = cons)
    df['items2'] = np.round(result.x, 0)
    return df, result



def objective(x):
    x=x[:,None]
#     print(pos.shape)
#     print((100*x*pc+pos).shape)
    a = (100*(x)*pc+pos)
#     b = (100*x*pc2+pos2)
#     print(a)
    a = np.sum(a, axis=0)
#     b = np.sum(b, axis=0)
#     a = a+b
#     print(a)
#     res = -(np.where(a<0,a,0))
#     res = -(a)
#     area = simps(res, dx=1)
    area = -np.trapz(a[a<0])
#     area = -np.trapz(-a[a<0])*(np.abs(max(a[a>0])))
#     return -(a[a<0].min())/100000
    r=np.sum(np.where((100*x*pc+pos)<0,1,0))/np.sum(np.where((100*x*pc+pos)>0,1,0))
#     return area/1000
#     return area
#     area = integrate.quad(a, 1.0*spot_min, 1.0*spot_max)
    return -area/10000

# objective(np.ones(36))
# cons = ({'type': 'ineq', 'fun': lambda x:  100- np.abs(x).sum(0)},
#        {'type': 'ineq', 'fun': lambda x:  4 + np.where(x>0,x,0).sum(0)\
#         - np.where(x<0,x,0).sum(0)})

# np.array_split(x, 2)
# define the starting point as a random sample from the domain
# pt = r_min + rand(2) * (r_max - r_min)
# perform the l-bfgs-b algorithm search

# df, dfp, pc, pos, pc2, pos2 =  getaux(df, bought, it, spot_min, spot_max, dbe)
# df, result = optimize_fn(df, bought, it,b_s_payoff_straddle, np.round(current_spot*0.2).astype('int'), np.round(current_spot*1.2).astype('int'))

# # df['items2']
# new_df = pd.DataFrame(np.round(result.x, 2),[df.contractName, df.theoretical])


# # new_df.head()
# print((np.array(np.round(result.x, 2))*df.theoretical).sum())
# print(result.success)
# new_df.reindex(new_df[0].abs().sort_values(ascending=False).index)
# dfp2=np.stack(df['bs_Payoff'])
# dfp2.shape

In [ ]:
from mip import *
import pulp as plp

def get_lb(i, df, current_spot, move):
    res = 0
    return res

def get_ub(i, df, current_spot, move):
    res =0
    if (df.option.iloc[i]=='C'):
        if move=='lc':
            res=1
        elif move == 'sc':
            if (((df.strike.iloc[i] < current_spot) )) :
                res = 0
            else:
                res=1      
    else:
        if move == 'lp':
            res=1
        elif move == 'sp':
            if ((df.strike.iloc[i] > current_spot) ) :
                res = 0
            else:
                res=1  
    return res
def get_theoretical(i, df):
    return df.theoretical.iloc[i]

def get_bs_theoretical(i, df, day, price):
    if not day:
        return df.theoretical.iloc[i]
    else:
        return df.bs.iloc[i][day][price]
    
def get_dzidx(df_stock, spot, price_range):
    danger_zone = range(round(spot - df_stock.Boli_int.iloc[-1]/2).astype('int'),\
                        round(spot + df_stock.Boli_int.iloc[-1]/2).astype('int') )
    dzidx= [price_range.index(x) for x in danger_zone]
    return dzidx, danger_zone

def get_weights(pc):
#     print(spot)
    a=(np.split(np.arange(pc[0].shape[0]),2))
    a1=(a[0]/len(a[0]))
    a2=(np.flip(a[1]-a[1][0])/len(a[1]))
    w=(np.concatenate([a1,a2]))
    return w

# get_weights(df_stock, current_spot, pc2_l)


In [ ]:
# # if x is Continuous
# x_vars  = {(i,j):
# plp.LpVariable(cat=plp.LpContinuous, 
#                lowBound=l[i,j], upBound=u[i,j], 
#                name="x_{0}_{1}".format(i,j)) 
# for i in set_I for j in set_J}
# # if x is Binary
# x_vars  = {(i,j):
# plp.LpVariable(cat=plp.LpBinary, name="x_{0}_{1}".format(i,j)) 
# for i in set_I for j in set_J}
# if x is Integer
n = (df.shape[0])
move_set = ['lc',  'lp', 'sc', 'sp']
it=4
bnd = it
def add_vars(bnd, df_contracts, current_spot, move_set):
    x_vars  = {(i,j):
    plp.LpVariable(cat=plp.LpInteger, 
                   lowBound=bnd*get_lb(i, df_contracts,current_spot, move_set[j]),\
                   upBound= bnd*get_ub(i, df_contracts,current_spot, move_set[j]),
                   name="x_"+df.contractName.iloc[i]+"_"+move_set[j]) 
    for i in range(df_contracts.shape[0]) for j in range(len(move_set)) }
    return x_vars


In [ ]:
# Less than equal constraints
def add_constr(x_vars, opt_model,dzidx,df_contracts, dfpl, dfps, pc_l, pc_s, pos, pc2_l, pc2_s, pc_final_l, pc_final_s, pos_final, pos2, day, price):
#     print(range(len(pc_l[0])))
    constraints_mid = {k : opt_model.addConstraint(
    plp.LpConstraint(
                 e=plp.lpSum(100*pc_l[i][k] * x_vars[i,j] for j in [0,1] for i in range(df_contracts.shape[0])) + 
                    plp.lpSum(+100*pc_s[i][k] * x_vars[i,j] for j in [2,3] for i in range(df_contracts.shape[0]))+
                    pos[k],
                 sense=plp.LpConstraintGE,
                 rhs=0,
                 name="constraint_mid_{0}".format(k)))
           for k in range(len(pc_l[0]))}

    constraints = {k : opt_model.addConstraint(
    plp.LpConstraint(
                 e=plp.lpSum(100*pc2_l[i][k] * x_vars[i,j] for j in [0,1] for i in range(df_contracts.shape[0])) + 
                    plp.lpSum(+100*pc2_s[i][k] * x_vars[i,j] for j in [2,3] for i in range(df_contracts.shape[0]))+
                    pos2[k],
                 sense=plp.LpConstraintGE,
                 rhs=0,
                 name="constraint_{0}".format(k)))
           for k in range(len(pc_l[0]))}

    constraints_final = {k : opt_model.addConstraint(
    plp.LpConstraint(
                 e=plp.lpSum(100*pc_final_l[i][k] * x_vars[i,j] for j in [0,1] for i in range(df_contracts.shape[0])) + 
                    plp.lpSum(+100*pc_final_s[i][k] * x_vars[i,j] for j in [2,3] for i in range(df_contracts.shape[0]))+
                    pos_final[k],
                 sense=plp.LpConstraintGE,
                 rhs=0,
                 name="constraint_final_{0}".format(k)))
           for k in range(len(pc_l[0])) }


    # >= constraints
    constraints_sclc = {0 : opt_model.addConstraint(
    plp.LpConstraint(
                 e=-plp.lpSum( x_vars[i,j] for j in [0] for i in range(df_contracts.shape[0])) + 
                    plp.lpSum( x_vars[i,j] for j in [2] for i in range(df_contracts.shape[0])) ,
                 sense=plp.LpConstraintLE,
                 rhs=2,
                 name="constraint_sc<lc+2"))
           }
    constraints_splp = {0 : opt_model.addConstraint(
    plp.LpConstraint(
                 e=-plp.lpSum( x_vars[i,j] for j in [1] for i in range(df_contracts.shape[0])) + 
                    plp.lpSum( x_vars[i,j] for j in [3] for i in range(df_contracts.shape[0])) ,
                 sense=plp.LpConstraintLE,
                 rhs=2,
                 name="constraint_sp<lp+2"))
           }
    constraints3 = {0 : opt_model.addConstraint(
    plp.LpConstraint(
                 e=+plp.lpSum( x_vars[i,j] for j in [0,1] for i in range(df_contracts.shape[0])) + 
                    plp.lpSum( x_vars[i,j] for j in [2,3] for i in range(df_contracts.shape[0])) ,
                 sense=plp.LpConstraintLE,
                 rhs=10,
                 name="constraint3"))
           }
    constraints4 = {0 : opt_model.addConstraint(
    plp.LpConstraint(
                 e=+plp.lpSum( 100*x_vars[i,j]*get_bs_theoretical(i, df_contracts, day, price) for j in [0,1] for i in range(df_contracts.shape[0])) ,
                 sense=plp.LpConstraintLE,
                 rhs=500,
                 name="constraint4"))
           }
    return constraints_mid, constraints,constraints_final,\
            constraints_sclc, constraints_splp, constraints3,\
            constraints4

# # == constraints
# constraints = {j : opt_model.addConstraint(
# plp.LpConstraint(
#              e=plp.lpSum(a[i,j] * x_vars[i,j] for i in set_I),
#              sense=plp.LpConstraintEQ,
#              rhs=b[j],
#              name="constraint_{0}".format(j)))
#        for j in set_J}

In [ ]:
# objective = plp.lpSum(plp.lpSum(100*pc2_l[i][k] * x_vars[i,j] for j in [0,1] for i in range(n)) + 
#                 plp.lpSum(+100*pc2_s[i][k] * x_vars[i,j] for j in [2,3] for i in range(n)) 
#                  +pos2[k] for k in range(len(pc_l[0])) )  / len(pc_l[0])  

def add_objective(x_vars,pc2_l, pc2_s, dzidx):
    w=get_weights(pc2_l)
    objective = plp.lpSum(plp.lpSum(100*w[k]*pc2_l[i][k] * x_vars[i,j] for j in [0,1] for i in range(df_contracts.shape[0])) + 
                plp.lpSum(+100*w[k]*pc2_s[i][k] * x_vars[i,j] for j in [2,3] for i in range(df_contracts.shape[0])) 
                 +pos2[k] for k in range(len(pc_l[0])) )  / len(range(len(pc_l[0])))
    return objective

# pc2_l.shape

In [ ]:
def get_model_results(x_vars, nm, df_contracts):
    opt_df = pd.DataFrame.from_dict(x_vars, orient="index", 
                                columns = ["variable_object"])
    opt_df.index = pd.MultiIndex.from_tuples(opt_df.index, 
                                   names=["column_i", "column_j"])
    opt_df.reset_index(inplace=True)
    # # Gurobi
    # opt_df["solution_value"] = 
    #   opt_df["variable_object"].apply(lambda item: item.X)
    # # CPLEX
    # opt_df["solution_value"] = 
    #   opt_df["variable_object"].apply(lambda item: item.solution_value)
    # PuLP
    opt_df["solution_value"] =  opt_df["variable_object"].apply(lambda item: item.varValue)
    opt_df.drop(columns=["variable_object"], inplace=True)
    opt_df.to_csv("./optimization_solution"+nm+".csv")
    # opt_df['column_j_'] = opt_df['column_j'].apply(lambda x: )
#     print(opt_df)
    sol = opt_df[opt_df['solution_value']>0]

#     df_contracts.contractName.iloc[sol['column_i'][:]], [move_set[i] for i in sol['column_j'].to_list()], a
    # a['items2'] = a.column_i.apply()
    sol['contractName']=sol.column_i.apply(lambda x: df_contracts.contractName.iloc[x] )
    sol['avg_price'] =sol.column_i.apply(lambda x: df_contracts.theoretical.iloc[x] )
#     print(sol)
    sol['items_l'] = sol.apply(lambda x: 1.0*x.solution_value if x.column_j in [0,1] else 0  , axis=1)
    sol['items_s'] = sol.apply(lambda x: 1.0*x.solution_value if x.column_j in [2,3] else 0  , axis=1)
    sol['items2'] = sol.apply(lambda x: -x.solution_value if x.column_j in [2,3] else x.solution_value, axis=1)
#     print(sol)
    df_contracts_ext = pd.merge(df_contracts,sol , how = 'left', on='contractName')
    df_contracts_ext['items_l'] = df_contracts_ext['items_l'].fillna(0)
    df_contracts_ext['items_s'] = df_contracts_ext['items_s'].fillna(0)
#     print(df_contracts_ext.iloc[0])
    df_contracts_ext['to_pay'] = df_contracts_ext.apply(lambda x: x.items_l*x.theoretical, axis=1)
    df_contracts_ext['long_pl_n'] =df_contracts_ext.apply(lambda x: 100*(x['items_l'])*x.long_pl, axis = 1)
    df_contracts_ext['short_pl_n'] =df_contracts_ext.apply(lambda x: 100*(x['items_s'])*x.short_pl, axis = 1)
    df_contracts_ext['items'] =df_contracts_ext.apply(lambda x: -x['items_s'] if x.column_j>=2 else x['items_l'], axis = 1)
    return sol, df_contracts_ext

In [ ]:
def optimizr(nm_model, bnd, df_contracts, current_spot, move_set,dfpl, dfps, pc_l, pc_s, pos, pc2_l, pc2_s, pc_final_l, pc_final_s, pos_final, pos2, dzidx, day, price ):
    opt_model = plp.LpProblem(name=nm_model)
    x_vars = add_vars(bnd, df_contracts, current_spot, move_set)
    constraints_mid, constraints,constraints_final,\
                constraints_sclc, constraints_splp, constraints3,\
                constraints4 = add_constr(x_vars,opt_model,dzidx,df_contracts, dfpl, dfps, pc_l, pc_s, pos, pc2_l, pc2_s, pc_final_l, pc_final_s, pos_final, pos2, day, price)
    objective=add_objective(x_vars,pc2_l, pc2_s, dzidx)
    # for maximization
    opt_model.sense = plp.LpMaximize
    # # for minimization
    # opt_model.sense = plp.LpMinimize
    opt_model.setObjective(objective)
    # solving with CBC
    sols=opt_model.solve()
#     print(opt_model)
    sol, df_contracts_ext = get_model_results(x_vars, nm_model, df_contracts)
    return opt_model, sol , df_contracts_ext, sols
    # solving with Glpk
    # opt_model.solve(solver = GLPK_CMD())

In [ ]:
dbe = df['daysBeforeExpiration'].iloc[0]-2
half_dbe = dbe//2
# objective function
df_cons_init = get_init_df(df_opts)
bought, b_s_payoff_straddle = get_bought(bought_real, df_cons_init, strike_max)
spot_min = np.round(current_spot*0.5).astype('int')
spot_max = np.round(current_spot*1.3).astype('int')
price_range = range(spot_min-1, spot_max)
dzidx, danger_zone = get_dzidx(df_stock, current_spot, price_range)
day = None
price= None
it =4
df_contracts, dfpl, dfps, pc_l, pc_s, pos, pc2_l, pc2_s, pc_final_l, pc_final_s, pos_final, pos2 =  getaux(df_cons_init, bought, it, spot_min, spot_max, dbe,  b_s_payoff_straddle)
print(1)
current_spot
nm_model = 'iter1'
opt_model, sol_prop, df_contracts_ext_prop, suxs = optimizr(nm_model, it, df_contracts, current_spot, move_set,dfpl, dfps, pc_l, pc_s, pos, pc2_l, pc2_s, pc_final_l, pc_final_s, pos_final, pos2, dzidx, day, price )
print(suxs)
bought_proposal = bought_real.copy()

bought_proposal.extend(sol_prop[['contractName', 'items2', 'avg_price' ]].values.tolist())
bought_prop, b_s_payoff_prop = get_bought(bought_proposal, df, strike_max)
bbdf =np.stack( df_contracts_ext_prop['long_pl_n'])
bbdf2 = np.stack(df_contracts_ext_prop['short_pl_n'])
# bbdf =np.stack(df['new_bs_Payoff'])

# new_bs_Payoff
# print(bought['bs_Payoff'][0][161])
b_s_to_bought_payoff_straddle_prop = np.sum(bbdf, axis=0)+np.sum(bbdf2, axis=0)

(640, 34)


AttributeError: 'int' object has no attribute 'astype'

In [ ]:
%matplotlib inline
from ipywidgets import interactive
payoff_straddle = np.sum(bought.optionPayoff)
theor_payoff_straddle = np.sum(bought.theor_optPayoff)
bb =np.stack(bought['bs_Payoff'])
# print(bought['bs_Payoff'][0][161])
b_s_payoff_straddle = np.sum(bb, axis=0)
# b_s_payoff_straddle=b_s_payoff_straddle[spot_min-1, spot_max]
# df['new_bs_Payoff'] =df.apply(lambda x: 100*np.abs(x['items2'])*call_bs_payoff (x['sT'], x['strike_price'], x['bs'],x['theoretical'], x['items2']  ) if x['option']=='C'  \
#                                      else 100*np.abs(x['items2'])*put_bs_payoff (x['sT'], x['strike_price'], x['bs'], x['theoretical'], x['items2'] ), axis = 1)





def f(future_stock_price, days_drom_today, sudden_time):
    fig, ax = plt.subplots()
    ax.spines['top'].set_visible(False) # Top border removed 
    ax.spines['right'].set_visible(False) # Right border removed
    ax.spines['bottom'].set_position('zero') # Sets the X-axis in the center
    x = np.linspace(-10, 10, num=1000)
    ax.plot(bought.sT.iloc[0],payoff_straddle,label='P_L_Final')
#     ax.plot(bought.sT.iloc[0],b_s_payoff_straddle[days_drom_today],label='B_S_Straddle')
    ax.plot(bought.sT.iloc[0],b_s_payoff_straddle[days_drom_today],label='P_L_per_Day')
    ax.plot(bought.sT.iloc[0],b_s_payoff_straddle[days_drom_today]+b_s_to_bought_payoff_straddle_prop[days_drom_today],label='P_L_After_Prop_per_Day')
#     ax.plot(bought.sT.iloc[0],+b_s_to_bought_payoff_straddle_prop[days_drom_today]\
#             + pl_holder_min[sudden_time][days_drom_today],label='P_L_After_Prop_per_Day_WhatIfLow', color = 'red', linestyle='dashed')
#     ax.plot(bought.sT.iloc[0],+b_s_to_bought_payoff_straddle_prop[days_drom_today]\
#             + pl_holder_max[sudden_time][days_drom_today],label='P_L_After_Prop_per_Day_WhatIfHigh', color='black', linestyle='dotted')
    plt.ylim(((b_s_payoff_straddle[days_drom_today]+b_s_to_bought_payoff_straddle_prop).min(),
              (b_s_payoff_straddle[days_drom_today]+b_s_to_bought_payoff_straddle_prop[days_drom_today]).max()))
    plt.ylim(-2500,2000)
    plt.xlim(np.round(current_spot*0.2).astype('int'), np.round(current_spot*1.4).astype('int'))
    plt.xlabel('Stock Price', ha='left')
    plt.ylabel('Profit and loss')
    plt.legend()
    plt.show()

    
interactive_plot = interactive(f, future_stock_price=(1, bought.S.iloc[0].max().astype(int)-1)\
                               , days_drom_today=(1, bought['days_array'].iloc[0].max().astype(int)-1, 1)\
                              , sudden_time=(1, bought['days_array'].iloc[0].max().astype(int)-1, 1))
output = interactive_plot.children[-1]
output.layout.height = '600px'
print('{} : {}'.format(spot_min, spot_max))
print('Current_price : {}'.format(current_spot))
print('Danger_Zone : {}'.format(danger_zone))
print('Amount Spent : {}'.format(df_contracts_ext.to_pay.sum(0)))
interactive_plot

22 : 57
Current_price : 43.49


NameError: name 'danger_zone' is not defined

In [ ]:
dzidx, danger_zone = get_dzidx(df_stock, spot, price_range)
df_contracts, dfpl, dfps, pc_l, pc_s, pos, pc2_l, pc2_s, pc_final_l, pc_final_s, pos_final, pos2 =  getaux(get_init_df(df_opts), bought, it, spot_min, spot_max, dbe)
current_spot
nm_model = 'iter1'
opt_model, a, df_contracts_ext, sols = optimizr(nm_model, it, df_contracts, current_spot, move_set,dfpl, dfps, pc_l, pc_s, pos, pc2_l, pc2_s, pc_final_l, pc_final_s, pos_final, pos2, dzidx )

In [ ]:

# bought

In [ ]:
sol_prop[['contractName', 'items2', 'avg_price' ]].values.tolist()

In [ ]:
payoff_straddle = np.sum(bought_prop.optionPayoff)
theor_payoff_straddle = np.sum(bought_prop.theor_optPayoff)
bb =np.stack(bought_prop['bs_Payoff'])
# print(bought['bs_Payoff'][0][161])
b_s_payoff_straddle = np.sum(bb, axis=0)
b_s_payoff_straddle[0].shape
# b_s_to_bought_payoff_straddle[0].shape
days_drom_today=10
bbdf =np.stack( df_contracts_ext['long_pl_n'])
bbdf2 = np.stack(df_contracts_ext['short_pl_n'])
# bbdf =np.stack(df['new_bs_Payoff'])

# new_bs_Payoff
# print(bought['bs_Payoff'][0][161])
b_s_to_bought_payoff_straddle = np.sum(bbdf, axis=0)+np.sum(bbdf2, axis=0)
# b_s_payoff_straddle[days_drom_today]+b_s_to_bought_payoff_straddle[days_drom_today]

In [ ]:
# bought.sT.iloc[0][dzidx]
# a= list(price_range)
danger_zone[0], danger_zone[-1]
# bought_prop.sT.iloc[0]
dbe = df['daysBeforeExpiration'].iloc[0]-2
half_dbe = dbe//2
# objective function
# bought_prop, b_s_payoff_prop = get_bought(bought_proposal, df, strike_max)
spot_min = np.round(current_spot*0.5).astype('int')
spot_max = np.round(current_spot*1.3).astype('int')
price_range = range(spot_min-1, spot_max)
dzidx, danger_zone = get_dzidx(df_stock, danger_zone[0], range(len(bought_prop.sT.iloc[0])))

it =4
df_contracts, dfpl, dfps, pc_l, pc_s, pos, pc2_l, pc2_s, pc_final_l, pc_final_s, pos_final, pos2 =  get_future_proposals(df_cons_init, bought_prop, it, spot_min, spot_max, dbe,  b_s_payoff_prop, pidx, didx)
print(1)
current_spot
nm_model = 'iter_day_'+str()
opt_model, sol, df_contracts_ext2, suxs = optimizr(nm_model, it, df_contracts, current_spot, move_set,dfpl, dfps, pc_l, pc_s, pos, pc2_l, pc2_s, pc_final_l, pc_final_s, pos_final, pos2, dzidx )

In [ ]:
pidx

In [ ]:
dzidx, danger_zone = get_dzidx(df_stock, current_spot, range(len(bought_prop.sT.iloc[0])))
dzidx, danger_zone,danger_zone[0]
dzidx2, danger_zone2 = get_dzidx(df_stock, danger_zone[0], range(len(bought_prop.sT.iloc[0])))
dzidx2, danger_zone2,danger_zone2[0]

In [ ]:
# bought_prop.sT.iloc[0]
dzidx, danger_zone = get_dzidx(df_stock, current_spot, range(len(bought_prop.sT.iloc[0])))
dzidx, danger_zone,danger_zone[0]
dbe = df_contracts['daysBeforeExpiration'].iloc[0]-2
a=[]
pl_holder_min=[]
pl_holder_max=[]

for i in range(0,dbe+1,1):
    dzidx2, danger_zone2 = get_dzidx(df_stock, current_spot, range(len(bought_prop.sT.iloc[0])))
#     print(dzidx2, danger_zone2)
    it =4
#     print(i)
    df_contracts, dfpl, dfps, pc_l, pc_s, pos, pc2_l, pc2_s, pc_final_l, pc_final_s, pos_final, pos2 =  get_future_proposals(df_cons_init, bought_prop, it, spot_min, spot_max, dbe,  b_s_payoff_prop,danger_zone2[0], i)
    t=pc_l
    #     print(1)
#     current_spot
    nm_model = 'prophet_low_day_'+str(i)
    opt_model, sol, df_contracts_ext, suxs = optimizr(nm_model, it, df_contracts, danger_zone[0], move_set,dfpl, dfps, pc_l, pc_s, pos, pc2_l, pc2_s, pc_final_l, pc_final_s, pos_final, pos2, dzidx2, i , danger_zone2[0] )
    
    bbdf =np.stack( df_contracts_ext['long_pl_n'])
    bbdf2 = np.stack(df_contracts_ext['short_pl_n'])
    # bbdf =np.stack(df['new_bs_Payoff'])

    # new_bs_Payoff
    # print(bought['bs_Payoff'][0][161])
    pl_holder_min.append(np.sum(bbdf, axis=0)+np.sum(bbdf2, axis=0))
    
    df_contracts, dfpl, dfps, pc_l, pc_s, pos, pc2_l, pc2_s, pc_final_l, pc_final_s, pos_final, pos2 =  get_future_proposals(df_cons_init, bought_prop, it, spot_min, spot_max, dbe,  b_s_payoff_prop,danger_zone2[-1], i)

    nm_model = 'prophet_high_day_'+str(i)
    opt_model, sol, df_contracts_ext, suxs = optimizr(nm_model, it, df_contracts, danger_zone[-1], move_set,dfpl, dfps, pc_l, pc_s, pos, pc2_l, pc2_s, pc_final_l, pc_final_s, pos_final, pos2, dzidx2, i, danger_zone2[-1] )
    
    bbdf =np.stack( df_contracts_ext['long_pl_n'])
    bbdf2 = np.stack(df_contracts_ext['short_pl_n'])
    # bbdf =np.stack(df['new_bs_Payoff'])

    # new_bs_Payoff
    # print(bought['bs_Payoff'][0][161])
    pl_holder_max.append(np.sum(bbdf, axis=0)+np.sum(bbdf2, axis=0))

In [ ]:
len(pl_holder_max), len(pl_holder_max[0])

In [ ]:
%matplotlib inline
from ipywidgets import interactive
payoff_straddle = np.sum(bought.optionPayoff)
theor_payoff_straddle = np.sum(bought.theor_optPayoff)
bb =np.stack(bought['bs_Payoff'])
# print(bought['bs_Payoff'][0][161])
b_s_payoff_straddle = np.sum(bb, axis=0)
# b_s_payoff_straddle=b_s_payoff_straddle[spot_min-1, spot_max]
# df['new_bs_Payoff'] =df.apply(lambda x: 100*np.abs(x['items2'])*call_bs_payoff (x['sT'], x['strike_price'], x['bs'],x['theoretical'], x['items2']  ) if x['option']=='C'  \
#                                      else 100*np.abs(x['items2'])*put_bs_payoff (x['sT'], x['strike_price'], x['bs'], x['theoretical'], x['items2'] ), axis = 1)





def f(future_stock_price, days_drom_today, sudden_time):
    fig, ax = plt.subplots()
    ax.spines['top'].set_visible(False) # Top border removed 
    ax.spines['right'].set_visible(False) # Right border removed
    ax.spines['bottom'].set_position('zero') # Sets the X-axis in the center
    x = np.linspace(-10, 10, num=1000)
    ax.plot(bought.sT.iloc[0],payoff_straddle,label='P_L_Final')
#     ax.plot(bought.sT.iloc[0],b_s_payoff_straddle[days_drom_today],label='B_S_Straddle')
    ax.plot(bought.sT.iloc[0],b_s_payoff_straddle[days_drom_today],label='P_L_per_Day')
    ax.plot(bought.sT.iloc[0],b_s_payoff_straddle[days_drom_today]+b_s_to_bought_payoff_straddle_prop[days_drom_today],label='P_L_After_Prop_per_Day')
    ax.plot(bought.sT.iloc[0],+b_s_to_bought_payoff_straddle_prop[days_drom_today]\
            + pl_holder_min[sudden_time][days_drom_today],label='P_L_After_Prop_per_Day_WhatIfLow', color = 'red', linestyle='dashed')
    ax.plot(bought.sT.iloc[0],+b_s_to_bought_payoff_straddle_prop[days_drom_today]\
            + pl_holder_max[sudden_time][days_drom_today],label='P_L_After_Prop_per_Day_WhatIfHigh', color='black', linestyle='dotted')
    plt.ylim(((b_s_payoff_straddle[days_drom_today]+b_s_to_bought_payoff_straddle_prop).min(),
              (b_s_payoff_straddle[days_drom_today]+b_s_to_bought_payoff_straddle_prop[days_drom_today]).max()))
    plt.ylim(-2500,2000)
    plt.xlim(np.round(current_spot*0.2).astype('int'), np.round(current_spot*1.4).astype('int'))
    plt.xlabel('Stock Price', ha='left')
    plt.ylabel('Profit and loss')
    plt.legend()
    plt.show()

    
interactive_plot = interactive(f, future_stock_price=(1, bought.S.iloc[0].max().astype(int)-1)\
                               , days_drom_today=(1, bought['days_array'].iloc[0].max().astype(int)-1, 1)\
                              , sudden_time=(1, bought['days_array'].iloc[0].max().astype(int)-1, 1))
output = interactive_plot.children[-1]
output.layout.height = '600px'
print('{} : {}'.format(spot_min, spot_max))
print('Current_price : {}'.format(current_spot))
print('Danger_Zone : {}'.format(danger_zone))
print('Amount Spent : {}'.format(df_contracts_ext.to_pay.sum(0)))
interactive_plot

In [ ]:
import re
df.contractName.iloc[a['column_i'][:]], [move_set[i] for i in a['column_j'].to_list()], a# a.name
b = [re.sub( '_s*$', '', y.name) for y in m.vars]
c = [ [y.x]+ y.name.split('_') for y in m.vars ]

c = [x for x in c if x[0] != 0]
c= [x for x in c if len(x)==3 ]


b = pd.DataFrame(c, columns = ['items2', 'contractName', 'move'])
b['items2'] = b.apply(lambda x: -1* x.items2 if x.move[0]=='s' else 1*x['items2'] , axis=1)
b
# # a.name
# df['items2'] = df.apply(lambda y: c[b.index(y.contractName)] if b.index(y.contractName) else 0 , axis=1)
# # m.vars.index('WFC210917P00047500')
df = pd.merge(df,a , how = 'left', on='contractName')
# df['items2'] = df['items2'].fillna(0)
# df[['items2', 'contractName']]

In [ ]:
pc_l[0]

In [ ]:
np.round(current_spot*0.2),np.round(current_spot*1.2)
current_spot
df[df['items2']!=0].head(1)

In [ ]:
# short_lim = df[['strike_price', 'option']]
# short_lim['current_spot'] = df_stock['Close'].iloc[-1]
# short_lim['short_lim'] = short_lim.apply(lambda x: -1 if (x.current_spot<x.strike_price and x.option=='C') or (x.current_spot>x.strike_price and x.option=='P') else 0, axis =1)
# short_lim['call_lim'] = 1
# short_lim['lim_bnds'] = short_lim.apply(lambda x: np.array([x.short_lim , x.call_lim]) , axis=1)
# ss = 20*np.array(short_lim.lim_bnds)
# np.stack(ss).shape
# s=short_lim.can_short.to_list()
# bnds*s


In [ ]:
# r_min, r_max = -20.0, 20
# bnds = np.tile([r_min, r_max], (pc.shape[0], 1))
# bnds.shape
# df.head(1)
df.columns

In [ ]:
b_s_to_bought_payoff_straddle.shape

In [ ]:
result.success

In [ ]:
current_spot

In [ ]:
pc

In [ ]:
x=np.array([0.2, 1.6])
np.where((x<0.5) & (x>0),1,0).sum()

In [ ]:
df_stock = get_stock_data(symbol, exchange, api_key=token, session=None)


In [ ]:
df_test = get_init_df(df_opts)

In [ ]:
df_test[df_test['contractName']=='WFC210917C00027500'].head()

In [ ]:
df_contracts_ext[df_contracts_ext.contractName=='WFC210917C00027500'].bs.iloc[0][136].shape

In [ ]:
lower, upper = 3.5, 6
mu, sigma = 5, 0.7
X = stats.truncnorm(
    (lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma)
N = stats.norm(loc=mu, scale=sigma)

fig, ax = plt.subplots(2, sharex=True)
ax[0].hist(X.rvs(10000), normed=True)
ax[1].hist(N.rvs(10000), normed=True)
plt.show()